# IMSE 660 Project IP Model - Nurse FTE Staffing Scenario 1
Brandon Thimmesch

5/8/2022

## Importing Libraries

In [1]:
import sys
import docplex.mp as mp
from docplex.mp.model import Model
import pandas as pd
import numpy as np

## Provided Information

In [2]:
Facility = ('Westside','Eastview','North Place','South Ridge')
Specialty = ('Cardiac', 'Orthopedic', 'General', 'Pediatric')

Month = ('January','February','March','April','May','June','July','August','September','October','November','December')

FS = [(i,j) for i in Facility for j in Specialty]
FSM = [(i,j,k) for i in Facility for j in Specialty for k in Month]

NurseDemand = {'Cardiac':{'January':50 ,'February':58 ,'March':40 ,'April':42 ,'May':30 ,'June':37 ,
                           'July':45 ,'August':46 ,'September':35 ,'October':36 ,'November':45 ,'December':44},
      'Orthopedic':{'January':63 ,'February':62 ,'March':55 ,'April':50 ,'May':48 ,'June':55 ,'July':60 ,
                  'August':65 ,'September':75 ,'October':80 ,'November':63 ,'December':65},
      'General':{'January':135 ,'February':155 ,'March':125 ,'April':120 ,'May':115 ,'June':120 ,
                     'July':100 ,'August':80 ,'September':95 ,'October':109 ,'November':116 ,'December':120},
      'Pediatric':{'January':70 ,'February':85 ,'March':75 ,'April':80 ,'May':60 ,'June':67 ,'July':78 ,
                     'August':68 ,'September':64 ,'October':70 ,'November':60 ,'December':45}}

NursePay = {'Westside':{'Cardiac':4550, 'Orthopedic':4450, 'General':4500, 'Pediatric':4650},
      'Eastview':{'Cardiac':4150, 'Orthopedic':4100, 'General':4450, 'Pediatric':4200},
      'North Place':{'Cardiac':3950, 'Orthopedic':3650, 'General':4100, 'Pediatric':3700},
      'South Ridge':{'Cardiac':4350, 'Orthopedic':4050, 'General':4050, 'Pediatric':4200}};

FacilityCapability = {'Westside':{'Cardiac':32000, 'Orthopedic':41000, 'General':35000, 'Pediatric':39000},
      'Eastview':{'Cardiac':29000, 'Orthopedic':32000, 'General':30000, 'Pediatric':26000},
      'North Place':{'Cardiac':26000, 'Orthopedic':28000, 'General':25000, 'Pediatric':27000},
      'South Ridge':{'Cardiac':32000, 'Orthopedic':34000, 'General':33000, 'Pediatric':31000}};

## IP Model Formulation: Scenario 1

In [3]:
class IPproblem:
    
        def __init__(self):
            self.m = Model(name = "Staffing Reccomendation: Scenario 1")
        
        def createVariables(self):
            self.yFS = self.m.binary_var_dict(keys = FS, name = 'Facility Speciality Selection', lb = 0)
            
            self.xFSM = self.m.integer_var_dict(keys = FSM, name = 'Nurses to be Scheduled', lb = 0)
                        
            self.cFSM = self.m.integer_var_dict(keys = FSM, name = 'Contract Nurses to be Scheduled', lb = 0)
            
        def createObjective(self):
            
            self.m.minimize(self.m.sum(self.yFS[i,j] * (FacilityCapability[i][j]) for i in Facility for j in Specialty)
                           + self.m.sum(self.xFSM[i,j,k] * (NursePay[i][j]) for i in Facility for j in Specialty for k in Month)
                           + self.m.sum(self.cFSM[i,j,k] * (1.2 * NursePay[i][j]) for i in Facility for j in Specialty for k in Month))           
            
        def createConstraints(self):
            # either south ridge OR east view offers pediatric surgery, not both
            self.m.add_constraint(self.yFS[('South Ridge','Pediatric')] + self.yFS[('Eastview','Pediatric')] == 1)
            
            # if North Place offers orthopedic surgery, must be offered at South Ridge too  
            self.m.add_constraint(self.yFS['North Place','Orthopedic'] <= self.yFS['South Ridge','Orthopedic'])
            
            # either Eastview or Westside must offer general surgery, possibly both
            self.m.add_constraint(self.yFS['Westside','General'] + self.yFS['Eastview','General'] >= 1)

            # number of nurses to staff at a facility in a specialty must be a minimum of 5
            for i in Facility:
                for j in Specialty:
                    for k in Month:
                        self.m.add_constraint(self.xFSM[i,j,k] >= 5*self.yFS[i,j])
            
            # do not staff nurses at facilities which are not open
            # number of nurses to staff for a facility specialty must not exceed 65
            for i in Facility:
                for j in Specialty:
                    for k in Month:
                        self.m.add_constraint(self.xFSM[i,j,k] / 65 <= self.yFS[i,j])
                        
            # number of nurses to staff must meet projected demand
            for j in Specialty:
                for k in Month:
                    self.m.add_constraint(self.m.sum(self.xFSM[i,j,k] + self.cFSM[i,j,k] for i in Facility) >= NurseDemand[j][k])
            
            # number of nurses to staff at a selected facility in a certain specialty must not deviate +- 20%
            for i in Facility:
                for j in Specialty:
                    for k in range(len(Month)-1):
                        self.m.add_constraint(self.xFSM[i,j,Month[k]] <= 1.2 * self.xFSM[i,j,Month[k+1]])
                        self.m.add_constraint(self.xFSM[i,j,Month[k]] >= 0.8 * self.xFSM[i,j,Month[k+1]])     
            
        def solveModel(self):
            self.m.solve()
            print(self.m.report)
            print("solve status =",self.m.get_solve_status()) 
            print(self.m.solution)

## Solution

In [4]:
solver = IPproblem()
solver.createVariables()
solver.createObjective()
solver.createConstraints()
solver.solveModel()

<bound method Model.report of docplex.mp.Model['Staffing Reccomendation: Scenario 1']>
solve status = JobSolveStatus.OPTIMAL_SOLUTION
solution for: Staffing Reccomendation: Scenario 1
objective: 1.38061e+07
Facility Speciality Selection_Eastview_General=1
Facility Speciality Selection_Eastview_Pediatric=1
Facility Speciality Selection_North Place_Cardiac=1
Facility Speciality Selection_North Place_Orthopedic=1
Facility Speciality Selection_North Place_General=1
Facility Speciality Selection_North Place_Pediatric=1
Facility Speciality Selection_South Ridge_Orthopedic=1
Facility Speciality Selection_South Ridge_General=1
Nurses to be Scheduled_Eastview_General_January=5
Nurses to be Scheduled_Eastview_General_February=6
Nurses to be Scheduled_Eastview_General_March=5
Nurses to be Scheduled_Eastview_General_April=5
Nurses to be Scheduled_Eastview_General_May=6
Nurses to be Scheduled_Eastview_General_June=7
Nurses to be Scheduled_Eastview_General_July=6
Nurses to be Scheduled_Eastview_Gene